In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv('updated_coughvid.csv')

In [2]:
df

,rmse,chroma_stft,spec_cent,spec_bw,rolloff,zcr,mfcc{1},mfcc{2},mfcc{3},mfcc{4},...,rapJitter,ppq5Jitter,ddpJitter,localShimmer,localdbShimmer,apq3Shimmer,aqpq5Shimmer,apq11Shimmer,ddaShimmer,label
0,0.065850,0.403767,1316.806414,1373.998076,2637.860622,0.057043,-396.59204,69.540160,2.152846,13.354017,...,0.021518,0.024053,0.064553,0.187378,1.718899,0.089158,0.143649,0.352439,0.267473,0
1,0.033997,0.532892,2474.234037,2125.162327,4869.731365,0.186172,-435.21085,45.288998,-12.166409,10.258451,...,0.014014,0.018379,0.042043,0.130333,1.313323,0.049385,0.059807,0.110768,0.148156,1
2,0.023310,0.370873,2158.381678,2007.817231,4750.294555,0.125032,-412.62552,54.555480,-1.768253,3.977824,...,0.033526,0.037981,0.100577,0.272967,2.125802,0.132889,0.196880,0.160610,0.398668,1
3,0.047035,0.479319,2678.491315,2139.232294,5136.450596,0.246256,-393.00226,48.030190,-29.901045,25.478853,...,0.032028,0.031223,0.096085,0.175612,1.617597,0.070595,0.094968,0.154762,0.211784,0
4,0.011785,0.741700,3316.010424,2345.969321,6007.887783,0.278178,-556.18726,10.974088,-10.207122,6.857212,...,0.023268,0.026566,0.069803,0.138516,1.320449,0.075879,0.081973,0.145956,0.227636,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24437,0.030827,0.429203,2974.741815,2265.905377,5435.183318,0.216093,-424.03302,38.712093,-16.247238,12.712377,...,0.029542,0.036789,0.088625,0.202130,1.619779,0.075903,0.199575,0.160610,0.227709,1
24438,0.030711,0.535591,2719.621677,2132.117936,5260.719083,0.198633,-471.09518,28.604359,-8.991700,12.397835,...,0.019939,0.022426,0.059816,0.176616,1.347352,0.107413,0.148694,0.117848,0.322239,1
24439,0.159654,0.389324,2360.664509,1696.391140,4114.219514,0.169187,-206.32933,59.016940,-74.789270,-1.210189,...,0.019389,0.020776,0.058168,0.164419,1.429379,0.093083,0.119234,0.175221,0.279249,0
24440,0.030981,0.451451,2788.705294,1800.585083,4708.568653,0.239603,-469.87784,59.399067,-41.076590,-2.028796,...,0.032847,0.036794,0.098542,0.205180,1.715662,0.105993,0.139264,0.166415,0.317979,1


In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten, Activation, Multiply, Permute, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from sklearn.preprocessing import MinMaxScaler

In [4]:
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

In [5]:
# Reshape the features to fit the CNN-LSTM model
X = X.reshape(X.shape[0], X.shape[1], 1)

# Define the input shape
input_shape = (X.shape[1], X.shape[2])


In [6]:
# Define the number of filters and kernel size for the convolutional layers
num_filters = 64
kernel_size = 3

# Define the number of units for the LSTM layer
num_units = 128

# Define the dropout rate
dropout_rate = 0.5

In [7]:
def attention_3d_block(inputs):
    # Compute the score by multiplying the input with a trainable weight vector
    attention_weights = Dense(1, activation='tanh')(inputs)
    # Reshape the score to a vector
    attention_weights = Flatten()(attention_weights)
    # Apply a softmax activation function to the score vector
    attention_weights = Activation('softmax')(attention_weights)
    # Reshape the score vector to a tensor
    attention_weights = Reshape((-1, 1))(attention_weights)
    # Multiply the input with the attention weights
    multiplied = Multiply()([inputs, attention_weights])
    # Compute the output of the attention mechanism
    output = K.sum(multiplied, axis=1)
    return output

In [8]:
# Define the model
input_layer = Input(shape=input_shape)
conv1 = Conv1D(num_filters, kernel_size, activation='relu')(input_layer)
maxpool1 = MaxPooling1D(pool_size=2)(conv1)
conv2 = Conv1D(num_filters, kernel_size, activation='relu')(maxpool1)
maxpool2 = MaxPooling1D(pool_size=2)(conv2)
lstm_layer = LSTM(num_units, return_sequences=True)(maxpool2)
attention_layer = attention_3d_block(lstm_layer)
dropout_layer = Dropout(dropout_rate)(attention_layer)
flatten_layer = Flatten()(dropout_layer)
output_layer = Dense(1, activation='sigmoid')(flatten_layer)
model = Model(inputs=input_layer, outputs=output_layer)

In [9]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [10]:
# Train the model
epochs = 20
batch_size = 32
history = model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.2)

loss, accuracy = model.evaluate(X, y)
print(f"Test accuracy: {accuracy}")

Epoch 1/20
612/612 [==============================] - 16s 21ms/step - loss: 0.6596 - accuracy: 0.6049 - val_loss: 0.5749 - val_accuracy: 0.7040
Epoch 2/20
612/612 [==============================] - 11s 18ms/step - loss: 0.5377 - accuracy: 0.7461 - val_loss: 0.5100 - val_accuracy: 0.7689
Epoch 3/20
612/612 [==============================] - 13s 21ms/step - loss: 0.4921 - accuracy: 0.7759 - val_loss: 0.4714 - val_accuracy: 0.8004
Epoch 4/20
612/612 [==============================] - 13s 21ms/step - loss: 0.4605 - accuracy: 0.7986 - val_loss: 0.5031 - val_accuracy: 0.7609
Epoch 5/20
612/612 [==============================] - 12s 19ms/step - loss: 0.4311 - accuracy: 0.8190 - val_loss: 0.4673 - val_accuracy: 0.8036
Epoch 6/20
612/612 [==============================] - 11s 18ms/step - loss: 0.4205 - accuracy: 0.8244 - val_loss: 0.4076 - val_accuracy: 0.8337
Epoch 7/20
612/612 [==============================] - 14s 22ms/step - loss: 0.4195 - accuracy: 0.8233 - val_loss: 0.4092 - val_accuracy:

In [11]:
# Define the model
input_layer = Input(shape=input_shape)
conv1 = Conv1D(num_filters, kernel_size, activation='relu')(input_layer)
maxpool1 = MaxPooling1D(pool_size=2)(conv1)
conv2 = Conv1D(num_filters, kernel_size, activation='relu')(maxpool1)
maxpool2 = MaxPooling1D(pool_size=2)(conv2)
conv3 = Conv1D(num_filters, kernel_size, activation='relu')(maxpool2)
maxpool3 = MaxPooling1D(pool_size=2)(conv3)
lstm_layer = LSTM(num_units, return_sequences=True)(maxpool3)
lstm_layer2 = LSTM(num_units, return_sequences=True)(lstm_layer)
attention_layer = attention_3d_block(lstm_layer2)
dropout_layer = Dropout(dropout_rate)(attention_layer)
flatten_layer = Flatten()(dropout_layer)
output_layer = Dense(1, activation='sigmoid')(flatten_layer)
model = Model(inputs=input_layer, outputs=output_layer)

In [12]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])



In [13]:
epochs = 20
batch_size = 32
history = model.fit(X, y, epochs=epochs, batch_size=batch_size, validation_split=0.2)

loss, accuracy = model.evaluate(X, y)
print(f"Test accuracy: {accuracy}")

Epoch 1/20
612/612 [==============================] - 25s 26ms/step - loss: 0.6489 - accuracy: 0.6076 - val_loss: 0.5349 - val_accuracy: 0.7492
Epoch 2/20
612/612 [==============================] - 14s 23ms/step - loss: 0.4866 - accuracy: 0.7851 - val_loss: 0.4424 - val_accuracy: 0.8130
Epoch 3/20
612/612 [==============================] - 15s 24ms/step - loss: 0.4345 - accuracy: 0.8163 - val_loss: 0.3914 - val_accuracy: 0.8386
Epoch 4/20
612/612 [==============================] - 15s 24ms/step - loss: 0.4087 - accuracy: 0.8331 - val_loss: 0.3882 - val_accuracy: 0.8437
Epoch 5/20
612/612 [==============================] - 15s 25ms/step - loss: 0.3973 - accuracy: 0.8397 - val_loss: 0.4037 - val_accuracy: 0.8323
Epoch 6/20
612/612 [==============================] - 14s 23ms/step - loss: 0.3824 - accuracy: 0.8450 - val_loss: 0.3618 - val_accuracy: 0.8556
Epoch 7/20
612/612 [==============================] - 14s 23ms/step - loss: 0.3719 - accuracy: 0.8502 - val_loss: 0.3693 - val_accuracy:

In [14]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(X, y, verbose=0)

# Predict the labels of the test set
y_pred = model.predict(X)

# Convert the predicted probabilities to labels
y_pred_labels = (y_pred > 0.5).astype(int)

# Calculate the performance metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix

accuracy = accuracy_score(y, y_pred_labels)
precision = precision_score(y, y_pred_labels)
recall = recall_score(y, y_pred_labels)
f1_score = f1_score(y, y_pred_labels)
roc_auc = roc_auc_score(y, y_pred)
tn, fp, fn, tp = confusion_matrix(y, y_pred_labels).ravel()
specificity = tn / (tn + fp)


764/764 [==============================] - 8s 9ms/step


In [15]:
print('Accuracy: ', accuracy)
print('Precision: ', precision)
print('Recall: ', recall)
print('f1_score',f1_score)
print('AUC: ', roc_auc)
print('Specificity: ', specificity)


Accuracy:  0.8752557073889207
Precision:  0.9407613097304177
Recall:  0.7621859767529059
f1_score 0.8421107141007717
AUC:  0.8938091233767842
Specificity:  0.9628285174967329


In [18]:
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Input, Dense, Dropout, LSTM, Embedding, Bidirectional, Conv1D, MaxPooling1D, Flatten, Activation, multiply, Lambda
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix



# Define the metrics to be calculated
metrics = {'accuracy': [], 'precision': [], 'recall': [], 'f1_score': [], 'roc_auc': []}

# Perform 3-fold cross-validation
kfold = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
for fold, (train, test) in enumerate(kfold.split(X, y)):
    print('Fold:', fold+1)
    
    # Define the model for this fold
    conv1 = Conv1D(num_filters, kernel_size, activation='relu')(input_layer)
    maxpool1 = MaxPooling1D(pool_size=2)(conv1)
    conv2 = Conv1D(num_filters, kernel_size, activation='relu')(maxpool1)
    maxpool2 = MaxPooling1D(pool_size=2)(conv2)
    conv3 = Conv1D(num_filters, kernel_size, activation='relu')(maxpool2)
    maxpool3 = MaxPooling1D(pool_size=2)(conv3)
    lstm_layer = LSTM(num_units, return_sequences=True)(maxpool3)
    lstm_layer2 = LSTM(num_units, return_sequences=True)(lstm_layer)
    attention_layer = attention_3d_block(lstm_layer2)
    dropout_layer = Dropout(dropout_rate)(attention_layer)
    flatten_layer = Flatten()(dropout_layer)
    output_layer = Dense(1, activation='sigmoid')(flatten_layer)
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # Train the model on the training data for this fold
    X_train, y_train = X[train], y[train]
    X_test, y_test = X[test], y[test]
    model.fit(X_train, y_train, epochs=30, batch_size=32)
    
    # Make predictions on the test data for this fold
    y_pred = model.predict(X_test)
    y_pred = np.round(y_pred).flatten()
    
    # Calculate the metrics for this fold and append to the list of metrics
    metrics['accuracy'].append(accuracy_score(y_test, y_pred))
    metrics['precision'].append(precision_score(y_test, y_pred))
    metrics['recall'].append(recall_score(y_test, y_pred))
    metrics['f1_score'].append(f1_score(y_test, y_pred))
    metrics['roc_auc'].append(roc_auc_score(y_test, y_pred))
    
    # Print the confusion matrix for this fold
    print('Confusion Matrix:')
    print(confusion_matrix(y_test, y_pred))
    print()

# Calculate and print the average metrics over all folds
for metric in metrics:
    print(metric, np.mean(metrics[metric]))


Fold: 1
Epoch 1/30
510/510 [==============================] - 18s 20ms/step - loss: 0.6389 - accuracy: 0.6249
Epoch 2/30
510/510 [==============================] - 11s 22ms/step - loss: 0.4979 - accuracy: 0.7762
Epoch 3/30
510/510 [==============================] - 11s 22ms/step - loss: 0.4615 - accuracy: 0.7985
Epoch 4/30
510/510 [==============================] - 13s 26ms/step - loss: 0.4311 - accuracy: 0.8198
Epoch 5/30
510/510 [==============================] - 13s 25ms/step - loss: 0.4073 - accuracy: 0.8334
Epoch 6/30
510/510 [==============================] - 10s 20ms/step - loss: 0.3941 - accuracy: 0.8416
Epoch 7/30
510/510 [==============================] - 11s 21ms/step - loss: 0.3814 - accuracy: 0.8482
Epoch 8/30
510/510 [==============================] - 12s 23ms/step - loss: 0.3755 - accuracy: 0.8514
Epoch 9/30
510/510 [==============================] - 11s 22ms/step - loss: 0.3695 - accuracy: 0.8541
Epoch 10/30
510/510 [==============================] - 12s 23ms/step - los

510/510 [==============================] - 10s 20ms/step - loss: 0.3380 - accuracy: 0.8674
Epoch 19/30
510/510 [==============================] - 11s 21ms/step - loss: 0.3325 - accuracy: 0.8707
Epoch 20/30
510/510 [==============================] - 10s 20ms/step - loss: 0.3308 - accuracy: 0.8722
Epoch 21/30
510/510 [==============================] - 11s 21ms/step - loss: 0.3319 - accuracy: 0.8717
Epoch 22/30
510/510 [==============================] - 14s 27ms/step - loss: 0.3247 - accuracy: 0.8750
Epoch 23/30
510/510 [==============================] - 13s 25ms/step - loss: 0.3277 - accuracy: 0.8731
Epoch 24/30
510/510 [==============================] - 12s 23ms/step - loss: 0.3216 - accuracy: 0.8752
Epoch 25/30
510/510 [==============================] - 10s 20ms/step - loss: 0.3248 - accuracy: 0.8735
Epoch 26/30
510/510 [==============================] - 10s 20ms/step - loss: 0.3200 - accuracy: 0.8771
Epoch 27/30
510/510 [==============================] - 11s 21ms/step - loss: 0.3169 -

In [19]:
print(metrics)

{'accuracy': [0.8559155621011291, 0.868295077942801, 0.8732048606849147], 'precision': [0.8851875808538163, 0.9486808818214673, 0.9827784156142365], 'recall': [0.7696850393700787, 0.7381889763779528, 0.7221597300337458], 'f1_score': [0.8234055354993983, 0.8303020718013601, 0.8325498460042146], 'roc_auc': [0.8461883385003703, 0.8536294478927446, 0.8561789719652502]}
